In [1]:
import glob
import nbformat
import os
import black
import isort

In [5]:
def isort_notebook_cells(notebook):
    with open(notebook, "rb") as fp:
        nb = nbformat.read(fp=fp, as_version=nbformat.NO_CONVERT)

    markdown_cells = list()
    code_cells = list()

    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            code_cells.append(cell)
        elif cell["cell_type"] == "markdown":
            markdown_cells.append(cell)
        else:
            raise Exception(cell["cell_type"])

    for code_cell in code_cells:
        if code_cell["source"] == "":
            continue
        if "import" in code_cell["source"]:
            s = isort.SortImports(file_contents=code_cell.source)
            code_cell.source = s.output.strip()

    with open(notebook, "w") as fp:
        nbformat.write(nb, fp)

In [6]:
def black_notebook_cells(notebook=None):
    with open(notebook, "rb") as fp:
        nb = nbformat.read(fp=fp, as_version=nbformat.NO_CONVERT)

    markdown_cells = list()
    code_cells = list()

    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            code_cells.append(cell)
        elif cell["cell_type"] == "markdown":
            markdown_cells.append(cell)
        else:
            raise Exception(cell["cell_type"])

    for code_cell in code_cells:
        if code_cell["source"] == "":
            continue
        try:
            code_cell["source"] = black.format_str(
                code_cell["source"],
                line_length=80
            )
        except:
            print("Failed: {}".format(code_cell["source"]))
            
    with open(notebook, "w") as fp:
        nbformat.write(nb, fp)

In [7]:
cleaning_functions = list()

cleaning_functions.append(black_notebook_cells)
cleaning_functions.append(isort_notebook_cells)

def format_folder(path=None):
    abs_path=os.path.abspath(path)
    try:
        assert os.path.exists(abs_path)
        notebooks = glob.glob(
            os.path.join(abs_path, "*.ipynb")
        )
        for notebook in notebooks:
            for cleaning_function in cleaning_functions:
                cleaning_function(
                    notebook=notebook
                )
    except:
        print(notebook)

In [8]:
notebook = "Bricks-Copy8.ipynb"

In [13]:
def clean_empty_notebook_cells(notebook):
    """ bubble imports to the top of a notebook"""
    with open(notebook, "rb") as fp:
        nb = nbformat.read(fp=fp, as_version=nbformat.NO_CONVERT)
    
    temp_cells = list()
       
    for cell in nb["cells"]:
        if cell["source"] == "":
            continue
        # Other side of the Valley.
        temp_cells.append(cell)

    nb["cells"]= temp_cells
    with open(notebook, "w") as fp:
        nbformat.write(nb, fp)

In [14]:
def bubble_import_notebook_cells(notebook):
    """ bubble imports to the top of a notebook"""
    with open(notebook, "rb") as fp:
        nb = nbformat.read(fp=fp, as_version=nbformat.NO_CONVERT)

    markdown_cells = list()
    code_cells = list()
    imports = list()
    
    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            code_cells.append(cell)
        elif cell["cell_type"] == "markdown":
            markdown_cells.append(cell)
        else:
            raise Exception(cell["cell_type"])

    for code_cell in code_cells:
        if code_cell["source"] == "":
            continue
        if "import" in code_cell["source"]:
            tmp_buffer = list()
            for line in code_cell["source"].splitlines():
                if "import" in line:
                    imports.append(line)
                else:
                    tmp_buffer.append(line)
            code_cell["source"] = tmp_buffer
    import_cell = nbformat.v4.new_code_cell()
    import_cell["source"]="\n".join(imports)
    nb["cells"].insert(0, import_cell)
    with open(notebook, "w") as fp:
        nbformat.write(nb, fp)

In [15]:
bubble_import_notebook_cells(notebook=notebook)
black_notebook_cells(notebook=notebook)
isort_notebook_cells(notebook=notebook)
clean_empty_notebook_cells(notebook=notebook)

Failed: class Square(GCode.Line):


In [16]:
cleaning_functions = list()

cleaning_functions.append(bubble_import_notebook_cells)
cleaning_functions.append(isort_notebook_cells)
cleaning_functions.append(black_notebook_cells)
cleaning_functions.append(clean_empty_notebook_cells)

In [32]:
!git add *.ipynb
!gitt add ../python_gcode/GCode/*.py
!git commit -am 'unclean.'

/bin/sh: 1: gitt: not found
On branch development/jed/submodule/CNC3018/2018-Jul
Your branch is up to date with 'origin/development/jed/submodule/CNC3018/2018-Jul'.

Untracked files:
	../blacken.sh
	../python_gcode/GCode/Program
	../python_gcode/GCode/Program.py

nothing added to commit but untracked files present


In [33]:
notebook = "Shapes-Copy3.ipynb"
for cleaning_function in cleaning_functions:
    cleaning_function(notebook=notebook)

In [35]:
!black --py36 ../python_gcode/GCode/*.py
!git commit -am 'clean.'

On branch development/jed/submodule/CNC3018/2018-Jul
Your branch is ahead of 'origin/development/jed/submodule/CNC3018/2018-Jul' by 1 commit.
  (use "git push" to publish your local commits)

Untracked files:
	../blacken.sh
	../python_gcode/GCode/Program
	../python_gcode/GCode/Program.py

nothing added to commit but untracked files present
Usage: black [OPTIONS] [SRC]...

  The uncompromising code formatter.

Options:
  -l, --line-length INTEGER       How many character per line to allow.
                                  [default: 88]
  --py36                          Allow using Python 3.6-only syntax on all
                                  input files.  This will put trailing commas
                                  in function signatures and calls also after
                                  *args and **kwargs.  [default: per-file
                                  auto-detection]
  --pyi                           Format all input files like typing stubs
                           